In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# EfficientNetB0 모델 불러오기 (ImageNet 가중치 사용)
base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
def extract_features(img_path):
    # 이미지 불러오기 및 전처리
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    # 특징 추출
    features = base_model.predict(img_array)
    return features
def calculate_similarity(features1, features2):
    # 코사인 유사도 계산
    return cosine_similarity(features1, features2)[0][0]

# 예제 이미지 경로
img_path1 = 'path_to_image1.jpg'
img_path2 = 'path_to_image2.jpg'
# 특징 추출
features1 = extract_features(img_path1)
features2 = extract_features(img_path2)
# 유사도 계산
similarity = calculate_similarity(features1, features2)
print(f"두 이미지의 유사도: {similarity}")
# 유사도가 임계값 이상이면 같은 객체로 간주
threshold = 0.8
if similarity > threshold:
    print("두 이미지는 같은 객체일 가능성이 높습니다.")
else:
    print("두 이미지는 다른 객체일 가능성이 높습니다.")


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.metrics.pairwise import cosine_similarity

# EfficientNetB0 모델 불러오기 (ImageNet 가중치 사용)
base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

def extract_features(img):
    # 이미지 전처리
    img_array = cv2.resize(img, (224, 224))  # 이미지 크기 조정
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    # 특징 추출
    features = base_model.predict(img_array)
    return features

def calculate_similarity(features1, features2):
    # 코사인 유사도 계산
    return cosine_similarity(features1, features2)[0][0]

def draw_matches(img1, img2, keypoints1, keypoints2, matches):
    # 매칭된 객체를 시각적으로 표시
    matched_img = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches, None)
    cv2.imshow("Matches", matched_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 예제 이미지 경로
img_path1 = 'path_to_image1.jpg'
img_path2 = 'path_to_image2.jpg'
# 이미지 불러오기
img1 = cv2.imread(img_path1)
img2 = cv2.imread(img_path2)
# SIFT를 사용하여 키포인트와 디스크립터 추출
sift = cv2.SIFT_create()
keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
keypoints2, descriptors2 = sift.detectAndCompute(img2, None)
# 매칭을 위한 BFMatcher 객체 생성
bf = cv2.BFMatcher()
matches = bf.knnMatch(descriptors1, descriptors2, k=2)
# 좋은 매칭 필터링
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

# 매칭된 객체 시각화
draw_matches(img1, img2, keypoints1, keypoints2, good_matches)

# 매칭된 객체들의 특징 추출 및 유사도 계산
for match in good_matches:
    img1_idx = match.queryIdx
    img2_idx = match.trainIdx

    # 매칭된 키포인트의 위치
    (x1, y1) = keypoints1[img1_idx].pt
    (x2, y2) = keypoints2[img2_idx].pt

    # 매칭된 객체의 특징 추출
    patch1 = img1[int(y1)-32:int(y1)+32, int(x1)-32:int(x1)+32]
    patch2 = img2[int(y2)-32:int(y2)+32, int(x2)-32:int(x2)+32]

    if patch1.shape == (64, 64, 3) and patch2.shape == (64, 64, 3):
        features1 = extract_features(patch1)
        features2 = extract_features(patch2)

        similarity = calculate_similarity(features1, features2)
        print(f"두 객체의 유사도: {similarity}")

        # 유사도가 임계값 이상이면 같은 객체로 간주
        threshold = 0.8
        if similarity > threshold:
            print("두 객체는 같은 객체일 가능성이 높습니다.")
        else:
            print("두 객체는 다른 객체일 가능성이 높습니다.")
